# Accessory genes related to least stable core genes

Since we found that least stable core genes are more co-expressed with accessory genes. Let's look at who those accessory genes are. In our [previous notebook](../3_core_core_analysis/4_find_related_acc_genes.ipynb) we annotated least and most stable core genes with their top co-expressed accessory gene.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import scipy
import pandas as pd
import numpy as np
from scripts import paths, utils

random.seed(1)

In [2]:
# Load transcriptional similarity df
# These are the subset of genes that we will consider
pao1_similarity_scores_filename = (
    "../3_core_core_analysis/pao1_core_similarity_associations_final_spell.tsv"
)
pa14_similarity_scores_filename = (
    "../3_core_core_analysis/pa14_core_similarity_associations_final_spell.tsv"
)

In [3]:
pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
# Get least stable core genes
pao1_least_stable_genes = list(
    pao1_similarity_scores[pao1_similarity_scores["label"] == "least stable"].index
)
pa14_least_stable_genes = list(
    pa14_similarity_scores[pa14_similarity_scores["label"] == "least stable"].index
)

The co-expressed accessory genes are listed in the `Related acc genes` column. The values are a list of accessory genes that were in the top 10 co-expressed genes, otherwise "No accessory genes"

In [5]:
pao1_least_df = pao1_similarity_scores.loc[pao1_least_stable_genes]

In [6]:
pa14_least_df = pa14_similarity_scores.loc[pa14_least_stable_genes]

In [7]:
pao1_least_df.head()

,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,mean expression,standard deviation expression,min expression,25% expression,50% expression,75% expression,max expression,variance expression,range expression,pathways present,Related acc genes
PAO1 id,,,,,,,,,,,,,,,,
PA0182,PA14_02300,0.126408,1.691876e-20,NaN,least stable,83.111864,421.695987,0.000000,18.610135,31.823490,61.014792,11811.693575,1.778275e+05,11811.693575,"['KEGG-Pathway-pae00780: Biotin metabolism', '...",No accessory genes
PA2222,PA14_59620,0.113817,6.858471e-17,NaN,least stable,393.111170,1226.692010,0.000000,51.183444,122.526860,233.600575,16615.976605,1.504773e+06,16615.976605,[],"['PA2224', 'PA2227', 'PA2221', 'PA2228', 'PA22..."
PA5185,PA14_68490,0.148277,1.123730e-27,NaN,least stable,96.393706,75.586970,0.000000,45.750026,74.597653,125.890384,632.779182,5.713390e+03,632.779182,[],No accessory genes
PA4883,PA14_64550,0.146946,3.318383e-27,NaN,least stable,68.166681,351.089538,0.000000,3.439621,7.993611,27.914230,6214.325606,1.232639e+05,6214.325606,[],No accessory genes
PA4598,PA14_60830,0.145670,9.283626e-27,mexD,least stable,602.482358,2087.480420,10.188671,99.920003,166.922821,412.896279,25997.893300,4.357575e+06,25987.704628,[],No accessory genes


## Concatenate accessory genes into a list

Note: We have to use `eval` here because we have a mix of strings and lists in our column of interest. In the future we could use an empty list instead of a string.

In [8]:
pao1_least_processed_df = pao1_least_df[
    pao1_least_df["Related acc genes"] != "No accessory genes"
]["Related acc genes"]
pa14_least_processed_df = pa14_least_df[
    pa14_least_df["Related acc genes"] != "No accessory genes"
]["Related acc genes"]

In [9]:
pao1_least_counts_df = (
    pd.Series(pao1_least_processed_df.apply(eval).sum())
    .value_counts()
    .to_frame("counts")
)
pa14_least_counts_df = (
    pd.Series(pa14_least_processed_df.apply(eval).sum())
    .value_counts()
    .to_frame("counts")
)

In [10]:
pao1_least_counts_df

,counts
PA2771,4
PA2225,4
PA1382,4
PA0135,3
PA0716,3
...,...
PA3499,1
PA2220,1
PA3066,1
PA3159,1


In [11]:
pa14_least_counts_df

,counts
PA14_43100,4
PA14_10830,4
PA14_54610,4
PA14_22280,3
PA14_35920,3
...,...
PA14_15470,1
PA14_46460,1
PA14_53680,1
PA14_35970,1


## Add gene names

In [12]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
pa14_annotation_filename = paths.GENE_PA14_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")
gene_mapping_pa14 = utils.get_pao1_pa14_gene_map(pa14_annotation_filename, "pa14")

In [13]:
pao1_gene_name_map = gene_mapping_pao1["Name"].to_frame()
pa14_gene_name_map = gene_mapping_pa14["Name"].to_frame()

In [14]:
gene_mapping_pao1.head()

,Name,Product.Name,GeneID.(PAO1),PA14_ID,annotation,num_mapped_genes
PAO1_ID,,,,,,
PA0001,dnaA,chromosomal replication initiator protein DnaA,878417.0,PA14_00010,core,1.0
PA0002,dnaN,"DNA polymerase III, beta chain",879244.0,PA14_00020,core,1.0
PA0003,recF,RecF protein,879229.0,PA14_00030,core,1.0
PA0004,gyrB,DNA gyrase subunit B,879230.0,PA14_00050,core,1.0
PA0005,lptA,"lysophosphatidic acid acyltransferase, LptA",877576.0,PA14_00060,core,1.0


In [15]:
# Add gene names
pao1_least_counts_df = pao1_least_counts_df.merge(
    gene_mapping_pao1["Name"], left_index=True, right_index=True, how="left"
)
pa14_least_counts_df = pa14_least_counts_df.merge(
    pa14_gene_name_map["Name"], left_index=True, right_index=True, how="left"
)

In [16]:
pao1_least_counts_df

,counts,Name
PA2771,4,NaN
PA2225,4,NaN
PA1382,4,NaN
PA0135,3,NaN
PA0716,3,NaN
...,...,...
PA3499,1,NaN
PA2220,1,NaN
PA3066,1,NaN
PA3159,1,wbpA


In [17]:
pa14_least_counts_df

,counts,Name
PA14_43100,4,rhsP2
PA14_10830,4,NaN
PA14_54610,4,NaN
PA14_22280,3,NaN
PA14_35920,3,NaN
...,...,...
PA14_15470,1,merP
PA14_46460,1,NaN
PA14_53680,1,NaN
PA14_35970,1,NaN


In [18]:
# Save
pao1_least_counts_df.to_csv("pao1_acc_coexpressed_with_least_stable.tsv", sep="\t")
pa14_least_counts_df.to_csv("pa14_acc_coexpressed_with_least_stable.tsv", sep="\t")